## Fine tune a Transformer Model like BERT

In [1]:
!pip install transformers -U

In [4]:
import pandas as pd
data=pd.read_csv("/content/train.csv", engine="python")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
data['toxic'].value_counts()

,count
toxic,
0,144277
1,15294


In [6]:
data=data[['comment_text','toxic']]
data=data[0:1000]
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer,BertForSequenceClassification

In [8]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
sample_data=["I am eating","I am playing"]
tokenizer(sample_data,padding=True,truncation=True,max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [10]:
model=model.to('cuda')

In [25]:
X=list(data['comment_text'])
y=list(data['toxic'])
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,stratify=y)
X_train_tokenized=tokenizer(X_train,padding=True,truncation=True, max_length=512)
X_val_tokenized=tokenizer(X_val,padding=True,truncation=True,max_length=512)

In [26]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [27]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [28]:
len(X_train), len(X_val)

(800, 200)

In [29]:
## Create a torch dataset
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels=None):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels']=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])


In [30]:
train_dataset=Dataset(X_train_tokenized,y_train)
val_dataset=Dataset(X_val_tokenized,y_val)

In [31]:
train_dataset[0]

{'input_ids': tensor([  101, 20228,  2015,  2507,  1037,  7514,  1010,  2748,  2030,  2053,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [41]:
def compute_metrics(p):
  print(type(p))
  pred,labels=p
  pred=np.argmax(pred,axis=1)

  accuracy=accuracy_score(y_true=labels,y_pred=pred)
  recall=recall_score(y_true=labels,y_pred=pred)
  precision=precision_score(y_true=labels,y_pred=pred)
  f1=f1_score(y_true=labels,y_pred=pred)

  return {"accuracy":accuracy,"recall":recall_score,"precision":precision_score,"f1":f1_score}

In [42]:
## Trainer
args=TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8
)

trainer=Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [43]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.08956565856933593, metrics={'train_runtime': 84.3365, 'train_samples_per_second': 9.486, 'train_steps_per_second': 1.186, 'total_flos': 210488844288000.0, 'train_loss': 0.08956565856933593, 'epoch': 1.0})

In [44]:
trainer.evaluate()

Trainer is attempting to log a value of "<function recall_score at 0x7dc0d4529300>" of type <class 'function'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "<function precision_score at 0x7dc0d45291c0>" of type <class 'function'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "<function f1_score at 0x7dc0d45289a0>" of type <class 'function'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.31939896941185,
 'eval_accuracy': 0.935,
 'eval_recall': <function sklearn.metrics._classification.recall_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>,
 'eval_precision': <function sklearn.metrics._classification.precision_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>,
 'eval_f1': <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>,
 'eval_runtime': 5.701,
 'eval_samples_per_second': 35.082,
 'eval_steps_per_second': 4.385,
 'epoch': 1.0}

In [45]:
np.set_printoptions(suppress=True)

In [47]:
text="That was a good point"
inputs=tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')
outputs=model(**inputs)
print(outputs)
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
print(predictions)

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.1217, -3.3877]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9985, 0.0015]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [48]:
trainer.save_model('CustomModel')

### Pushing the model to HuggingFace Hub

In [50]:
!pip install huggingface_hub

In [51]:
model=BertForSequenceClassification.from_pretrained('CustomModel')

In [52]:
from huggingface_hub import notebook_login
notebook_login()

In [54]:
model.push_to_hub("SachdevaVansh/FinetuneBERT")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SachdevaVansh/FinetuneBERT/commit/48cfb2c252021c0516155821b0a2abc58a3657bb', commit_message='Upload BertForSequenceClassification', commit_description='', oid='48cfb2c252021c0516155821b0a2abc58a3657bb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SachdevaVansh/FinetuneBERT', endpoint='https://huggingface.co', repo_type='model', repo_id='SachdevaVansh/FinetuneBERT'), pr_revision=None, pr_num=None)

In [55]:
model=BertForSequenceClassification.from_pretrained("SachdevaVansh/FinetuneBERT")

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
model